# Biolib hackaton challenge partition

In [2]:
import h5py
import numpy as np
import pandas as pd

In [2]:
data = h5py.File("/home/eryk/development/NSPThesis/data/nsp3/training_data/TS115_ESM1b.npz", 'r')

In [4]:
idx_encoding = np.arange(0, 20)
idx_embed = np.arange(68, 1348)
idx_mask = np.arange(50, 51)
idx_ss = np.arange(57, 65)

classes = np.concatenate([idx_encoding, idx_embed, idx_mask, idx_ss])

In [4]:
p = sorted(np.random.permutation(len(data['dataset']))[:1000])

In [5]:
with h5py.File('/home/eryk/development/NSPThesis/data/chunked/Train_ESM1b.hdf5', 'w') as f:
    d = f.create_dataset('dataset', (1000, 1632, 1309), chunks=True, compression="gzip", compression_opts=9)

    step=100
    for i in range(0, 1000, step):
        d[i:i+step] = np.nan_to_num(data['dataset'][p[i:i+step]][:, :, classes])

In [6]:
data = h5py.File("../../data/nsp3/training_data/CASP12_HHblits_ESM1b.hdf5", 'r')

In [7]:
with h5py.File('/home/eryk/development/NSPThesis/data/chunked/CASP12_ESM1b.hdf5', 'w') as f:
    d = f.create_dataset('dataset', (21, 1494, 1309), chunks=True, compression="gzip", compression_opts=9)

    d[:] = np.nan_to_num(data['dataset'][:][:, :, classes])

In [8]:
np.savez_compressed("/home/eryk/development/NSPThesis/data/nsp3/training_data/TS115_ESM1b.npz", dataset=data['dataset'][:][:, :, classes])

In [9]:
np.load("/home/eryk/development/NSPThesis/data/nsp3/training_data/TS115_ESM1b.npz")

In [3]:
test = np.load("/home/eryk/development/NSPThesis/data/nsp3/training_data/CASP12_ESM1b.npz")['data']

In [7]:
# create predictions.csv
Q8, Q3 = "GHIBESTC", "HEC"


structure_mask = np.array([0, 0, 0, 1, 1, 2, 2, 2])

q8 = np.array([Q8[int(val)] for val in np.argmax(test[:, :, 1301:1309], axis=2).flatten()])
q3 = np.array([Q3[int(val)] for val in np.amax(test[:, :, 1301:1309] * structure_mask, axis=2).flatten()])

mask = test[:, :, 1300].flatten().astype(int)

q8 = q8[mask == 1]
q3 = q3[mask == 1]

lengths = np.array([sum(row) for row in test[:, :, 1300]])

lengths = np.pad(lengths, (0, len(q8)-len(lengths)), mode='constant')

df = np.concatenate([np.expand_dims(q8, axis=1), np.expand_dims(q3, axis=1), np.expand_dims(lengths, axis=1)], axis=1)

# save to file
df = pd.DataFrame(df)
df = df.set_axis(["q8", "q3", "lengths"], axis=1, inplace=False)
df.to_csv("predictions.csv")

In [47]:
np.savez_compressed("/home/eryk/development/NSPThesis/data/nsp3/training_data/TS115_ESM1b_2.npz", data=test[p])

In [44]:
np.load("/home/eryk/development/NSPThesis/data/nsp3/training_data/CASP12_ESM1b.npz")['data'].shape

(21, 1494, 1309)